# IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Report
### *Opening a New Shopping Mall in New Delhi, India*

* Build a dataframe of neighborhoods in New Delhi, Malaysia by web scraping the data from Wikipedia page
* Get the geographical coordinates of the neighborhoods
* Obtain the venue data for the neighborhoods from Foursquare API
* Explore and cluster the neighborhoods
* Select the best cluster to open a new shopping mall


### 1. Import libraries

In [160]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#!pip install geocoder
import geocoder

#!pip install lxml
#!pip install beautifulsoup4
#import requests
#!pip install beautifulsoup4 
from bs4 import BeautifulSou

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [96]:
data = requests.get("https://en.wikipedia.org/wiki/Neighbourhoods_of_Delhi").text

In [97]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [98]:
# create a list to store neighborhood data
neighborhoodList = []

In [99]:
for row in soup.find_all("div", class_="mw-content-ltr")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [100]:
df = pd.DataFrame({"Neighborhood": neighborhoodList})


In [104]:
for i in range(202,2595):
    df = df.drop(index=[i],axis=0)    
for i in range(0,12):
    df = df.drop(index=[i],axis=0)

In [117]:
df = df.drop(index=[114],axis=0)
df = df.drop(index=[148],axis=0)
df = df.drop(index=[107],axis=0)

In [118]:
df.shape

(184, 3)

In [119]:
df = df.reset_index(drop=True)

### 3. Get the geographical coordinates

In [120]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Delhi, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [121]:
coords = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]

In [122]:
print(len(coords))

184


In [123]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [124]:
df

,Neighborhood,Latitude,Longitude
0,Adarsh Nagar,28.720340,77.172660
1,Ashok Vihar,28.690420,77.176060
2,Azadpur,28.712420,77.173110
3,Bawana,28.800590,77.034730
4,Begum Pur,28.732600,77.052170
5,Dhaka,28.707084,77.204518
6,Jahangirpuri,28.728760,77.175640
7,Karala,28.737310,77.039630
8,Keshav Puram,28.688010,77.158660
9,Kingsway Camp,28.711690,77.202050


In [125]:
df.to_csv("Dl_df", index=False)

### 4. Create a map of New Delhi with neighborhoods superimposed on top

In [126]:
address = 'Delhi, India'

geolocator = Nominatim(user_agent="dl-explr")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Delhi, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi, India 28.6517178, 77.2219388.


In [127]:
map_dl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_dl)  
    
map_dl

In [128]:
# save the map as HTML file
map_dl.save('map_dl.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [129]:
# define Foursquare Credentials and Version
CLIENT_ID = 'DXBPYZ1L3PJYIJA2T0DYHPDCABWDA2RRXD4E25KZOGZKDDA5' # your Foursquare ID
CLIENT_SECRET = 'QISOLSFGFCFLR5VGBH0KC5ASDXAM25NKURYQTP3AWUXRNFAB' # your Foursquare Secret
VERSION = '20191107' # Foursquare API version
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DXBPYZ1L3PJYIJA2T0DYHPDCABWDA2RRXD4E25KZOGZKDDA5
CLIENT_SECRET:QISOLSFGFCFLR5VGBH0KC5ASDXAM25NKURYQTP3AWUXRNFAB


#### Now, let's get the top 100 venues that are within a radius of 1000 meters.

In [130]:
radius = 1000

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [131]:

# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2757, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adarsh Nagar,28.72034,77.17266,Axis Bank ATM,28.723032,77.170631,ATM
1,Adarsh Nagar,28.72034,77.17266,Adarsh Nagar Metro Station,28.716598,77.170436,Light Rail Station
2,Adarsh Nagar,28.72034,77.17266,Vishyavidyalaya Metro Station@Entry gate #1 n ...,28.715596,77.170981,Train Station
3,Adarsh Nagar,28.72034,77.17266,Bandhu Dental Clinic,28.714134,77.167224,Spa
4,Ashok Vihar,28.69042,77.17606,Bellagio,28.696361,77.180021,Asian Restaurant


In [60]:

# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2880, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adarsh Nagar,28.72034,77.17266,Axis Bank ATM,28.723032,77.170631,ATM
1,Adarsh Nagar,28.72034,77.17266,Adarsh Nagar Metro Station,28.716598,77.170436,Light Rail Station
2,Adarsh Nagar,28.72034,77.17266,Vishyavidyalaya Metro Station@Entry gate #1 n ...,28.715596,77.170981,Train Station
3,Adarsh Nagar,28.72034,77.17266,Bandhu Dental Clinic,28.714134,77.167224,Spa
4,Ashok Vihar,28.69042,77.17606,Bellagio,28.696361,77.180021,Asian Restaurant


#### Let's check how many venues were returned for each neighorhood

In [132]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adarsh Nagar,4,4,4,4,4,4
Alaknanda,10,10,10,10,10,10
Anand Vihar,5,5,5,5,5,5
Ashok Nagar,7,7,7,7,7,7
Ashok Vihar,11,11,11,11,11,11
Azadpur,7,7,7,7,7,7
Babarpur,6,6,6,6,6,6
Badarpur,6,6,6,6,6,6
Bali Nagar,20,20,20,20,20,20


#### Let's find out how many unique categories can be curated from all the returned venues

In [133]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 210 uniques categories.


In [134]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['ATM', 'Light Rail Station', 'Train Station', 'Spa',
       'Asian Restaurant', 'Pizza Place', 'Sandwich Place',
       'South Indian Restaurant', 'Snack Place', 'Indian Restaurant',
       'Market', 'Dessert Shop', 'Coffee Shop', 'Department Store',
       'Bus Station', 'Park', 'Restaurant', 'Playground',
       'Mobile Phone Shop', 'Tourist Information Center', 'Food Truck',
       'Café', 'Korean Restaurant', 'Burger Joint', 'Electronics Store',
       'Golf Course', 'Vegetarian / Vegan Restaurant', 'Diner',
       'Athletics & Sports', 'Fast Food Restaurant', 'Basketball Court',
       'Shopping Mall', 'Multiplex', 'Donut Shop', 'Frozen Yogurt Shop',
       'Ice Cream Shop', "Men's Store", 'Chinese Restaurant', 'Lake',
       'Bakery', 'Convenience Store', 'Furniture / Home Store', 'Hotel',
       'Clothing Store', 'Bank', 'Photography Studio', 'Metro Station',
       'Museum', 'Mosque', 'Monument / Landmark'], dtype=object)

In [64]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [135]:
# one hot encoding
Dl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Dl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Dl_onehot.columns[-1]] + list(Dl_onehot.columns[:-1])
Dl_onehot = Dl_onehot[fixed_columns]

print(Dl_onehot.shape)
Dl_onehot.head()

(2757, 211)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Breakfast Spot,Bridal Shop,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,English Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hardware Store,Health & Beauty Service,Hindu Temple,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Northeast Indian Restaurant,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Punjabi Restaurant,Rental Car Location,Resort,Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supplement Shop,Tailor Shop,Tapas Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tibetan Restaurant,Tourist Information Center,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Adarsh Nagar,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [136]:

Dl_grouped = Dl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(Dl_grouped.shape)
Dl_grouped

(172, 211)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Breakfast Spot,Bridal Shop,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,English Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hardware Store,Health & Beauty Service,Hindu Temple,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Northeast Indian Restaurant,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Punjabi Restaurant,Rental Car Location,Resort,Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supplement Shop,Tailor Shop,Tapas Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tibetan Restaurant,Tourist Information Center,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Adarsh Nagar,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000

In [137]:
len(Dl_grouped[Dl_grouped["Shopping Mall"] > 0])

28

#### Create a new DataFrame for Shopping Mall data only

In [138]:
Dl_mall = Dl_grouped[["Neighborhoods","Shopping Mall"]]

In [139]:
Dl_mall.head()

,Neighborhoods,Shopping Mall
0,Adarsh Nagar,0.0
1,Alaknanda,0.0
2,Anand Vihar,0.0
3,Ashok Nagar,0.0
4,Ashok Vihar,0.0


### 7. Cluster Neighborhoods
##### Run k-means to cluster the neighborhoods in New Delhi into 3 clusters.

In [148]:
# set number of clusters
kclusters = 3

Dl_clustering = Dl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Dl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 2, 0], dtype=int32)

In [149]:

# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
dl1_merged = Dl_mall.copy()

# add clustering labels
dl1_merged["Cluster Labels"] = kmeans.labels_

In [150]:

dl1_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
dl1_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Adarsh Nagar,0.0,0
1,Alaknanda,0.0,0
2,Anand Vihar,0.0,0
3,Ashok Nagar,0.0,0
4,Ashok Vihar,0.0,0


In [152]:

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dl1_merged = dl1_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(dl1_merged.shape)
dl1_merged.head() # check the last columns!

(182, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Adarsh Nagar,0.0,0,28.72034,77.17266
106,Nizamuddin West,0.0,0,28.58973,77.24522
107,Okhla,0.0,0,28.53249,77.27838
108,Old Delhi Railway Station,0.0,0,28.66748,77.23200
109,Paharganj,0.0,0,28.64596,77.21493


In [153]:
# sort the results by Cluster Labels
print(dl1_merged.shape)
dl1_merged.sort_values(["Cluster Labels"], inplace=True)
dl1_merged

(182, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Adarsh Nagar,0.000000,0,28.720340,77.172660
33,Friends Colony,0.000000,0,28.674780,77.305320
21,Dariba Kalan,0.000000,0,28.654568,77.233419
20,"Dabri, New Delhi",0.000000,0,28.607590,77.087210
19,Connaught Place,0.000000,0,28.633920,77.219780
1,Alaknanda,0.000000,0,28.525680,77.253240
2,Anand Vihar,0.000000,0,28.653560,77.313730
3,Ashok Nagar,0.000000,0,28.692230,77.301270
4,Ashok Vihar,0.000000,0,28.690420,77.176060
5,Azadpur,0.000000,0,28.712420,77.173110


#### Finally, let's visualize the resulting clusters

In [154]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dl1_merged['Latitude'], dl1_merged['Longitude'], dl1_merged['Neighborhood'], dl1_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [155]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters
##### Cluster 0

In [157]:
dl1_merged.loc[dl1_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Adarsh Nagar,0.000000,0,28.720340,77.172660
33,Friends Colony,0.000000,0,28.674780,77.305320
21,Dariba Kalan,0.000000,0,28.654568,77.233419
20,"Dabri, New Delhi",0.000000,0,28.607590,77.087210
19,Connaught Place,0.000000,0,28.633920,77.219780
1,Alaknanda,0.000000,0,28.525680,77.253240
2,Anand Vihar,0.000000,0,28.653560,77.313730
3,Ashok Nagar,0.000000,0,28.692230,77.301270
4,Ashok Vihar,0.000000,0,28.690420,77.176060
5,Azadpur,0.000000,0,28.712420,77.173110


##### Cluster 1

In [158]:
dl1_merged.loc[dl1_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
133,Sadatpur,0.333333,1,28.721700,77.265140
25,Delhi Cantonment,0.250000,1,28.591530,77.129420
141,Sarai Rohilla,0.200000,1,28.665830,77.184810
49,Inderpuri,0.181818,1,28.651070,77.306690
153,Silampur,0.250000,1,28.670110,77.271220
74,Kotwali,0.200000,1,28.598079,77.128204
78,Laxmi Nagar,0.333333,1,28.638750,77.275920


##### Cluster 2

In [159]:
dl1_merged.loc[dl1_merged['Cluster Labels'] == 2]


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
152,Shivaji Place,0.047619,2,28.652650,77.121390
143,Sarojini Nagar,0.041667,2,28.575700,77.193620
143,Sarojini Nagar,0.041667,2,28.575700,77.193620
91,Moti Nagar,0.090909,2,28.656720,77.129600
58,Kabir Nagar,0.125000,2,28.689676,77.140972
89,Model Town,0.062500,2,28.705010,77.189500
66,Keshav Puram,0.062500,2,28.688010,77.158660
158,Tihar Village,0.076923,2,28.635450,77.107780
100,Nehru Place,0.052632,2,28.600750,77.292480
102,Netaji Nagar,0.042553,2,28.577470,77.185160



Observations:
Most of the shopping malls are concentrated in the central area of New Delhi city, with the highest number in cluster 2 and moderate number in cluster 0. On the other hand, cluster 1 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.